In [57]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as mp
import pandas as pd
import seaborn as sns
import sys
import os
sys.path.append('/Users/vs/Dropbox/Python')
import astropy.io.votable
### From Gaia documentation to do command line requests to Gaia DR1 archive
import httplib
import urllib
import time
from xml.dom.minidom import parseString

sns.set_style("white")
sns.set_context("paper", font_scale=1.5, rc={"lines.linewidth": 2.0})
sns.set_palette(sns.husl_palette(10, l=.4))
colors = sns.color_palette()


In [2]:
def run_gaia_query(adql_query, outputFileName):
    host = "gea.esac.esa.int"
    port = 80
    pathinfo = "/tap-server/tap/async"
    headers = {"Content-type": "application/x-www-form-urlencoded", "Accept":       "text/plain"}
    params = urllib.urlencode({"REQUEST": "doQuery", "LANG": "ADQL", "FORMAT": "csv", "PHASE":  "RUN", "QUERY": adql_query})

    connection = httplib.HTTPConnection(host, port)
    connection.request("POST",pathinfo,params,headers)

    #Status
    response = connection.getresponse()
    print "Status: " +str(response.status), "Reason: " + str(response.reason)

    #Server job location (URL)
    location = response.getheader("location")
    print "Location: " + location

    #Jobid
    jobid = location[location.rfind('/')+1:]
    print "Job id: " + jobid

    connection.close()

    #-------------------------------------
    #Check job status, wait until finished

    while True:
        connection = httplib.HTTPConnection(host, port)
        connection.request("GET",pathinfo+"/"+jobid)
        response = connection.getresponse()
        data = response.read()
        #XML response: parse it to obtain the current status
        dom = parseString(data)
        phaseElement = dom.getElementsByTagName('uws:phase')[0]
        phaseValueElement = phaseElement.firstChild
        phase = phaseValueElement.toxml()
        print "Status: " + phase
        #Check finished
        if phase == 'COMPLETED': break
        #wait and repeat
        time.sleep(0.2)

    #print "Data:"
    #print data

    connection.close()

    #-------------------------------------
    #Get results
    connection = httplib.HTTPConnection(host, port)
    connection.request("GET",pathinfo+"/"+jobid+"/results/result")
    response = connection.getresponse()
    data = response.read()
    outputFile = open(outputFileName, "w")
    outputFile.write(data)
    outputFile.close()
    connection.close()
    print "Data saved in: " + outputFileName
    return(0)



## Grab all the info about the CRRP sources

In [3]:
## CRRP stars:

crrp_stars = pd.read_csv('vizer_crossmatch.tsv', skiprows=166, skipinitialspace=True, names=('input', 'rad', 'HIP', 'TYC2', 'SolID', 'Source', 'RandomI', 'Epoch', 'RA_ICRS', 'e_RA_ICRS', 'DE_ICRS', 'e_DE_ICRS', 'Plx', 'e_Plx', 'pmRA', 'e_pmRA', 'pmDE', 'e_pmDE', 'RADEcor', 'RAPlxcor', 'RApmRAcor', 'RApmDEcor', 'DEPlxcor', 'DEpmRAcor', 'DEpmDEcor', 'PlxpmRAcor', 'PlxpmDEcor', 'pmRApmDEcor', 'NAL', 'NAC', 'NgAL', 'NgAC', 'NbAL', 'NbAC', 'DQ', 'epsi', 'sepsi', 'APF', 'ARF', 'WAL', 'WAC', 'Apr', 'MatchObs', 'Dup', 'sK1', 'sK2', 'sK3', 'sK4', 'mK1', 'mK2', 'mK3', 'mK4', 'o_<Gmag>', '<FG>', 'e_<FG>', '<Gmag>', 'Var', 'GLON', 'GLAT', 'ELON', 'ELAT'), na_values='NOT_AVAILABLE', sep=';', comment='#')
crrp_stars['ID'] = crrp_stars.input.str.split('\t',0).str.get(0)

crrp_stars = crrp_stars.drop('input', 1)
#crrp_stars = crrp_stars.drop('rad', 1)
crrp_stars = crrp_stars.replace('', np.nan)
crrp_stars['ID'] = crrp_stars['ID'].replace(regex=True, to_replace=r' ',value='_')
crrp_stars['id_compare'] = map(str.lower, crrp_stars.ID)
crrp_stars['id_compare'] = crrp_stars['id_compare'].replace(regex=True, to_replace=r'_',value='')
crrp_stars = crrp_stars.groupby(['ID']).min()


In [4]:
info_df = pd.read_csv('rrl_average_mags', delim_whitespace=True)


In [5]:
merged_df = info_df.merge(crrp_stars, on='id_compare')

In [6]:
merged_df

,Name,Period,Type,id_compare,mag_3p6,err_3p6,amp_3p6,mag_4p5,err_4p5,amp_4p5,...,mK4,o_<Gmag>,<FG>,e_<FG>,<Gmag>,Var,GLON,GLAT,ELON,ELAT
0,ABUma,0.600,ab,abuma,9.598,0.003,0.171,9.587,0.003,0.177,...,9.267754,105,8.257196e+05,7685.37020,10.733,NaN,141.042099,67.861413,158.465090,43.874357
1,AMTuc,0.406,c,amtuc,10.602,0.002,0.127,10.565,0.002,0.142,...,-17.344803,124,3.928950e+05,4741.63750,11.539,NaN,299.059083,-49.006000,324.481499,-64.212527
2,ANSer,0.522,ab,anser,9.801,0.004,0.289,9.795,0.004,0.299,...,-19.213884,212,6.546377e+05,10730.71765,10.985,NaN,23.802692,45.232103,232.759715,32.403224
3,AVPeg,0.390,ab,avpeg,9.332,0.004,0.281,9.329,0.004,0.281,...,30.296535,85,1.103026e+06,29572.87476,10.418,NaN,77.443012,-24.053691,339.290110,33.146829
4,BHPeg,0.641,ab,bhpeg,9.002,0.003,0.237,8.982,0.003,0.236,...,31.870094,104,1.159717e+06,14555.13569,10.364,NaN,85.614589,-38.356182,350.987445,21.094589
5,BXLeo,0.363,c,bxleo,10.678,0.002,0.107,10.670,0.002,0.122,...,-15.991236,70,4.162361e+05,5720.86812,11.476,NaN,241.078445,69.984066,168.310156,12.988380
6,CSEri,0.311,c,cseri,8.126,0.002,0.117,8.110,0.002,0.117,...,44.986927,193,4.172558e+06,47371.36513,8.974,NaN,256.317812,-63.387143,15.201311,-54.053147
7,DXDel,0.473,ab,dxdel,8.653,0.004,0.249,8.641,0.004,0.258,...,24.143590,78,1.771644e+06,13135.50447,9.904,NaN,58.469794,-18.844886,318.268456,29.160267
8,HKPup,0.734,ab,hkpup,9.884,0.004,0.252,9.854,0.004,0.257,...,-30.420750,111,5.056835e+05,6778.87633,11.265,NaN,230.724295,5.543249,121.137119,-33.749926
9,MTTel,0.317,c,mttel,8.078,0.002,0.126,8.064,0.002,0.111,...,35.748550,45,4.282056e+06,66644.82592,8.946,NaN,350.239779,-21.117388,281.605877,-23.839117


## just using the CRRP data we have 39 stars using Spitzer
## can we add in the wise data for stars in the variable star region where there are Gaia + allWISE light curves so there is a bigger sample?

In [7]:
adql_query = "SELECT * from gaiadr1.allwise_original_valid WHERE allwise_oid IN (SELECT allwise_oid FROM gaiadr1.allwise_best_neighbour WHERE source_id IN (SELECT source_id FROM gaiadr1.gaia_source WHERE (phot_variable_flag='VARIABLE')))"

run_gaia_query(adql_query, 'variable_stars_allwise_photometry.csv')

adql_query = "SELECT * FROM gaiadr1.allwise_best_neighbour WHERE source_id IN (SELECT source_id FROM gaiadr1.gaia_source WHERE (phot_variable_flag='VARIABLE'))"
run_gaia_query(adql_query, 'variable_stars_allwise.csv')

Status: 303 Reason: 303
Location: http://gea.esac.esa.int/tap-server/tap/async/1476812145715O
Job id: 1476812145715O
Status: EXECUTING
Status: EXECUTING
Status: COMPLETED
Data saved in: variable_stars_allwise_photometry.csv
Status: 303 Reason: 303
Location: http://gea.esac.esa.int/tap-server/tap/async/1476812147812O
Job id: 1476812147812O
Status: COMPLETED
Data saved in: variable_stars_allwise.csv


0

In [8]:
variables_df = pd.read_csv('variable_stars.csv', header=0, sep=',')
variables_extended = pd.read_csv('variable_stars_sourceinfo.csv', header=0, sep=',')

variables_df = variables_df.merge(variables_extended, on='source_id')
variables_df = variables_df.rename(columns={'solution_id_x':'solution_id_var', 'solution_id_y': 'solution_id_source'})
variables_df['Period'] = variables_df.apply(lambda x: 1./x.phot_variable_fundam_freq1, axis=1)


In [9]:
allwise_df = pd.read_csv('variable_stars_allwise_photometry.csv', header=0, sep=',')
allwise_df_var = pd.read_csv('variable_stars_allwise.csv', header=0, sep=',')
allwise_df = allwise_df.merge(allwise_df_var, on='allwise_oid')


In [10]:
wise_rrl_df = variables_df.merge(allwise_df, on='source_id')
wise_rrl_df = wise_rrl_df[wise_rrl_df.w1gmag!=np.nan]
wise_rrl_df = wise_rrl_df[wise_rrl_df.classification=='RRLYR']

wise_rrl_df = wise_rrl_df.drop(['ra_y', 'dec_y'], 1)
wise_rrl_df = wise_rrl_df.rename(columns={'ra_x': 'ra', 'dec_x': 'dec'})

## Not trying to do a crossmatch to the allwise time series photometry right now. will do an arbitraty cut on period for the RRab/RRc's to guess the classifications of the ones I don't know


In [11]:
useful = ['source_id', 'classification', 'Period', 'ra', 'dec', 'l', 'b', 'parallax', 'parallax_error', 'phot_g_mean_mag']
useful_crrp = ['Name', 'Source', 'Type', 'Period', 'RA_ICRS', 'DE_ICRS', 'GLON', 'GLAT', 'Plx', 'e_Plx', '<Gmag>']

In [12]:
combined_sample_df = merged_df[useful_crrp]
combined_sample_df = combined_sample_df.rename(columns={'Source': 'source_id', 'Type': 'classification', 'RA_ICRS': 'ra', 'DE_ICRS': 'dec', 'GLON': 'l', 'GLAT': 'b', 'Plx': 'parallax', 'e_Plx': 'parallax_error', '<Gmag>': 'phot_g_mean_mag'})


In [13]:
combined_sample_df = pd.concat([combined_sample_df, wise_rrl_df[useful]], ignore_index=True)
combined_sample_df = combined_sample_df.reset_index(drop=True)

In [14]:
combined_sample_df

,Name,Period,b,classification,dec,l,parallax,parallax_error,phot_g_mean_mag,ra,source_id
0,ABUma,0.600000,67.861413,ab,47.828763,141.042099,0.93,0.27,10.733000,182.810685,1546016668386865792
1,AMTuc,0.406000,-49.006000,c,-67.918161,299.059083,0.85,0.26,11.539000,19.627973,4692528057537147136
2,ANSer,0.522000,45.232103,ab,12.961105,23.802692,0.77,0.29,10.985000,238.379390,1191509999055192960
3,AVPeg,0.390000,-24.053691,ab,22.574791,77.443012,1.53,0.23,10.418000,328.011708,1793460110951463424
4,BHPeg,0.641000,-38.356182,ab,15.787682,85.614589,1.40,0.22,10.364000,343.254215,2828497064068310784
5,BXLeo,0.363000,69.984066,c,16.543330,241.078445,0.53,0.28,11.476000,174.508478,3972712532526824448
6,CSEri,0.311000,-63.387143,c,-42.963311,256.317812,2.16,0.23,8.974000,39.274539,4947090013255935616
7,DXDel,0.473000,-18.844886,ab,12.464108,58.469794,1.66,0.22,9.904000,311.868205,1760981190300823808
8,HKPup,0.734000,5.543249,ab,-13.098978,230.724295,0.53,0.26,11.265000,116.195095,3030561875047012352
9,MTTel,0.317000,-21.117388,c,-46.653841,350.239779,1.43,0.31,8.946000,285.550322,6662886601414152448


In [63]:
def gaia_to_pandas(adql_query):
    host = "gea.esac.esa.int"
    port = 80
    pathinfo = "/tap-server/tap/async"
    headers = {"Content-type": "application/x-www-form-urlencoded", "Accept":       "text/plain"}
    params = urllib.urlencode({"REQUEST": "doQuery", "LANG": "ADQL", "FORMAT": "csv", "PHASE":  "RUN", "QUERY": adql_query})

    connection = httplib.HTTPConnection(host, port)
    connection.request("POST",pathinfo,params,headers)

    #Status
    response = connection.getresponse()
    print "Status: " +str(response.status), "Reason: " + str(response.reason)

    #Server job location (URL)
    location = response.getheader("location")
    print "Location: " + location

    #Jobid
    jobid = location[location.rfind('/')+1:]
    print "Job id: " + jobid

    connection.close()

    #-------------------------------------
    #Check job status, wait until finished

    while True:
        connection = httplib.HTTPConnection(host, port)
        connection.request("GET",pathinfo+"/"+jobid)
        response = connection.getresponse()
        data = response.read()
        #XML response: parse it to obtain the current status
        dom = parseString(data)
        phaseElement = dom.getElementsByTagName('uws:phase')[0]
        phaseValueElement = phaseElement.firstChild
        phase = phaseValueElement.toxml()
        print "Status: " + phase
        #Check finished
        if phase == 'COMPLETED': break
        #wait and repeat
        time.sleep(0.2)

    #print "Data:"
    #print data

    connection.close()

    #-------------------------------------
    #Get results
    connection = httplib.HTTPConnection(host, port)
    connection.request("GET",pathinfo+"/"+jobid+"/results/result")
    response = connection.getresponse()
    data = response.read()
    outputFile = open('table.csv', "w")
    outputFile.write(data)
    outputFile.close()
    temp = pd.read_csv('table.csv', header=0, sep=',')
    connection.close()
    #data = pd.read_json(data)
    return(temp)



In [19]:
combined_sample_df[np.isfinite(combined_sample_df.parallax)==False].source_id

40     4675338120949840768
41     4663209992299609088
42     4663197863313540736
43     4673212322590824064
44     4662677656874460032
45     4677006011368736384
46     4676957117461314432
47     4663020086028882816
48     4659803636559757056
49     4676532396735780608
50     4674783864009351040
51     4662838838404846464
52     4662774345176819072
53     4663061317714625792
54     4660772654203382656
55     4673214109297259776
56     4675234491978981760
57     4675973947907563904
58     4662654910725565440
59     4662877802347897472
60     4675168177683130496
61     4662511733699815296
62     4662647523381657088
63     4674974457479225088
64     4674099211863344256
65     4662871102199022080
66     4672891260900478464
67     4663050734911261440
68     4673191908611877632
69     4675505349795989632
              ...         
265    4663256206146756224
266    4663158658851089920
267    4663090282972426112
268    4663477036186953088
269    4663000466618458752
270    4663207449678915840
2

In [64]:
star_id = '4675338120949840768'
adql_query = "SELECT parallax, parallax_error FROM gaiadr1.gaia_source WHERE (source_id={0})".format(star_id)
data = gaia_to_pandas(adql_query)

Status: 303 Reason: 303
Location: http://gea.esac.esa.int/tap-server/tap/async/1476813917622O
Job id: 1476813917622O
Status: COMPLETED


In [65]:
data

,parallax,parallax_error
0,NaN,NaN


In [66]:
adql_query

'SELECT parallax, parallax_error FROM gaiadr1.gaia_source WHERE (source_id=4675338120949840768)'

In [71]:
variables_df[np.isfinite(variables_df.parallax)==True]

,solution_id_var,source_id,phot_variable_fundam_freq1,classification,solution_id_source,random_index,ref_epoch,ra,ra_error,dec,...,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_mag,phot_variable_flag,l,b,ecl_lon,ecl_lat,Period
2416,374678695396246029,5283957629860435072,1.752518,RRLYR,1635378410781933568,444118735,2015.0,91.94041,0.129556,-66.977428,...,516,313636.500838,2817.124822,11.783704,VARIABLE,276.873805,-29.051596,209.455272,-89.128589,0.570608
